### Setup

In [1]:
import gc
import os
import pandas as pd

from src.base import Base
from src.gbrt import GBRT
from src.transformer import BaseTRF, GBRT_TRF
from src.utils import load_pickle
from test import aida_global, aida_local

In [2]:
EMB_PATH = os.path.join(os.getcwd(), 'embeddings')

embs  = ["wiki2vec_w10_100d.pkl", "word2vec-google-news-300",
         "glove-wiki-gigaword-300", "fasttext-wiki-news-subwords-300"]

entities = load_pickle('./data/aida/entities.pkl')
entities_filtered = load_pickle('./data/aida/entities_filtered.pkl')
entities_full = load_pickle('./data/aida/entities_full.pkl')

## Test Baseline Models

### Local Context

Investigate the effects of cased vs uncased

In [3]:
results = []
for emb in embs[1:]:
    res = [emb]
    model = Base(os.path.join(EMB_PATH, emb))
    model.entity_desc_dict = entities
    for case in [True, False]:
        model.cased = case
        acc, _ = aida_local(model)
        res.append(acc)
    results.append(res)
    model = None
    gc.collect()

# Wikipedia2Vec 100D
acc, _ = aida_local(Base(os.path.join(EMB_PATH, embs[0])))
results.append([embs[0], None, acc])

# Transformer
model = BaseTRF()
model.entity_desc_dict = entities
acc, _ = aida_local(model)
results.append(['Transformer Base', acc, None])

# Results
pd.DataFrame(results, columns=['Embedding', 'Accuracy (cased)', 'Accuracy (uncased)'])

100%|██████████| 231/231 [18:41<00:00,  4.86s/it]


,Embedding,Accuracy (cased),Accuracy (uncased)
0,word2vec-google-news-300,51.461,44.808
1,glove-wiki-gigaword-300,51.566,51.285
2,fasttext-wiki-news-subwords-300,43.647,42.344
3,wiki2vec_w10_100d.pkl,NaN,65.118
4,Transformer Base,52.165,NaN


Investigate the effects of keeping nouns only

In [4]:
results = []
for emb in embs:
    model = Base(os.path.join(EMB_PATH, emb), nouns_only=True,
                 cased=(emb in [embs[1], embs[3]]))
    model.entity_desc_dict = entities
    acc, _ = aida_local(model)
    results.append([emb, acc])
    model = None
    gc.collect()

# Results
pd.DataFrame(results, columns=['Embedding', 'Accuracy (Nouns Only)'])

100%|██████████| 231/231 [02:46<00:00,  1.39it/s]


,Embedding,Accuracy (Nouns Only)
0,wiki2vec_w10_100d.pkl,62.619
1,word2vec-google-news-300,51.109
2,glove-wiki-gigaword-300,49.243
3,fasttext-wiki-news-subwords-300,45.759


### Global Context

In [5]:
results = []
for emb in embs:
    model = Base(os.path.join(EMB_PATH, emb), cased=(emb in [embs[1], embs[3]]))
    model.entity_desc_dict = entities
    acc, _ = aida_local(model, use_document=True)
    results.append([emb, acc])
    model = None
    gc.collect()

# Transformer
model = BaseTRF()
model.entity_desc_dict = entities
acc, _ = aida_local(model, use_document=True)
results.append(['Transformer Base', acc])

# Results
pd.DataFrame(results, columns=['Embedding', 'Accuracy (Context = Whole Document)'])

100%|██████████| 231/231 [22:26<00:00,  5.83s/it] 


,Embedding,Accuracy (Context = Whole Document)
0,wiki2vec_w10_100d.pkl,57.709
1,word2vec-google-news-300,57.214
2,glove-wiki-gigaword-300,56.850
3,fasttext-wiki-news-subwords-300,41.334
4,Transformer Base,57.115


Investigating the effects of using global context with nouns only and using whole wikipedia page to encode an entity

In [6]:
results = []
for emb in embs[1:]:
    model = Base(os.path.join(EMB_PATH, emb), nouns_only=True,
                 cased=(emb in [embs[1], embs[3]]))
    model.entity_desc_dict = entities_filtered
    acc, _ = aida_local(model, use_document=True)
    results.append([emb, acc])
    model = None
    gc.collect()

# Results
pd.DataFrame(results, columns=['Embedding', 'Accuracy (Nouns Only, whole document, full desc)'])

100%|██████████| 231/231 [20:00<00:00,  5.20s/it]


,Embedding,"Accuracy (Nouns Only, whole document, full desc)"
0,word2vec-google-news-300,69.660
1,glove-wiki-gigaword-300,63.123
2,fasttext-wiki-news-subwords-300,57.016


In [5]:
model = BaseTRF()
model.entity_desc_dict = entities_full
acc, _ = aida_local(model, use_document=True)
print("Accuracy:", acc)

100%|██████████| 231/231 [1:21:44<00:00, 21.23s/it]

Accuracy: 57.214


## Test GBRT (and it's variations)

In [7]:
results = []
pretrained = ['base.pkl', 'string_sim.pkl', 'context.pkl']
for i in pretrained:
    model = GBRT(os.path.join(EMB_PATH, embs[0]), model_path=i)
    acc, res = aida_global(model)
    results.append([i[:-4], acc])
    model = None
    gc.collect()

pd.DataFrame(results, columns=['Model', 'Accuracy'])

100%|██████████| 231/231 [03:56<00:00,  1.02s/it]


,Model,Accuracy
0,base,84.913
1,string_sim,86.794
2,context,89.435


In [3]:
embs.insert(1, 'wiki2vec_w10_300d.pkl')

results = []
for emb in embs:
    model = GBRT(os.path.join(EMB_PATH, emb), model_path=f"{emb}_trained.pkl",
                 cased=(emb in [embs[2], embs[4]]))
    model.entity_desc_dict = entities_filtered
    res = []
    for x in [False, True]:
        model.two_step = x
        acc, _ = aida_global(model)
        res.append(acc)
    results.append([emb, *res])
    model = None
    gc.collect()
    
pd.DataFrame(results, columns=['Model', 'Accuracy', 'Accuracy (with two-step)'])

100%|██████████| 231/231 [30:09<00:00,  7.83s/it] 


,Model,Accuracy,Accuracy (with two-step)
0,wiki2vec_w10_100d.pkl,92.539,92.704
1,wiki2vec_w10_300d.pkl,91.680,91.416
2,word2vec-google-news-300,92.572,92.572
3,glove-wiki-gigaword-300,91.482,91.350
4,fasttext-wiki-news-subwords-300,88.577,87.290


In [7]:
model = GBRT_TRF(ranker_path="TRF_trained.pkl")
res = []
for x in [False, True]:
    model.two_step = x
    model.entity_desc_dict = entities
    acc, _ = aida_global(model)
    res.append(acc)

print(f"Transformer GBRT\nAccuracy\t\t{res[0]}\nAccuracy (two-step)\t{res[1]}")

Transformer GBRT
Accuracy		89.601
Accuracy (two-step)	89.634
